# Three-Phase Forecast Model - Product Sales
## Модель прогнозування продажів товарів (3 фази)

In [ ]:
# Імпорт бібліотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
import warnings
warnings.filterwarnings('ignore')

print('✅ Бібліотеки імпортовано успішно!')

## Фаза 1: Завантаження та підготовка даних

In [ ]:
# Завантаження датасету
df = pd.read_csv('dataset_pcs.csv')

print('📊 Датасет завантажено!')
print(f'Розмір: {df.shape}')
print(f'\nПерші рядки:')
print(df.head())
print(f'\nІнформація про датасет:')
print(df.info())

In [ ]:
# Перетворення дати
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Додаткові ознаки
df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['day_of_year'] = df['date'].dt.dayofyear

print('✅ Дані підготовлено!')
print(f'Період даних: {df["date"].min()} - {df["date"].max()}')

In [ ]:
# Розділення на Phase 1 (2022-2023), Phase 2 (2024 Q1-Q3), Phase 3 (2024 Q4 - прогноз)
phase1_data = df[df['date'] < '2024-01-01'].copy()
phase2_data = df[(df['date'] >= '2024-01-01') & (df['date'] < '2024-10-01')].copy()
phase3_data = df[df['date'] >= '2024-10-01'].copy()

print('📅 Дані розділено на фази:')
print(f'Phase 1 (Training): {len(phase1_data)} записів ({phase1_data["date"].min()} - {phase1_data["date"].max()})')
print(f'Phase 2 (Validation): {len(phase2_data)} записів ({phase2_data["date"].min()} - {phase2_data["date"].max()})')
print(f'Phase 3 (Forecast): {len(phase3_data)} записів ({phase3_data["date"].min()} - {phase3_data["date"].max()})')

## Фаза 2: Навчання моделі

In [ ]:
# Вибір ознак для моделі
feature_columns = [
    'product_id', 'day_of_week', 'month', 'is_weekend',
    'price', 'discount_percent', 'stock_level', 
    'competitor_price', 'quarter', 'day_of_year'
]

target_column = 'sales_quantity'

# Підготовка даних для навчання (Phase 1)
X_train = phase1_data[feature_columns]
y_train = phase1_data[target_column]

# Підготовка даних для валідації (Phase 2)
X_val = phase2_data[feature_columns]
y_val = phase2_data[target_column]

print('✅ Ознаки підготовлено!')
print(f'Кількість ознак: {len(feature_columns)}')
print(f'Ознаки: {feature_columns}')

In [ ]:
# Нормалізація даних
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print('✅ Дані нормалізовано!')

In [ ]:
# Навчання Three-Phase Linear моделі
model = LinearRegression()
model.fit(X_train_scaled, y_train)

print('✅ Модель three_phase_linear навчена!')
print(f'\nКоефіцієнти моделі:')
coef_df = pd.DataFrame({
    'Feature': feature_columns,
    'Coefficient': model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)
print(coef_df)

In [ ]:
# Збереження моделі
with open('three_phase_linear.pkl', 'wb') as f:
    pickle.dump({'model': model, 'scaler': scaler, 'features': feature_columns}, f)

print('✅ Модель збережена у файл: three_phase_linear.pkl')

## Фаза 3: Прогнозування та оцінка

In [ ]:
# Прогноз на Phase 2 (валідація)
y_pred_val = model.predict(X_val_scaled)

# Прогноз на Phase 3 (майбутнє)
X_test = phase3_data[feature_columns]
X_test_scaled = scaler.transform(X_test)
y_pred_test = model.predict(X_test_scaled)

print('✅ Прогнози згенеровано!')
print(f'Phase 2 predictions: {len(y_pred_val)}')
print(f'Phase 3 predictions: {len(y_pred_test)}')

In [ ]:
# Створення результуючого DataFrame для Phase 3
forecast_df = phase3_data[['date', 'product_id', 'product_category']].copy()
forecast_df['actual_sales'] = phase3_data[target_column].values
forecast_df['predicted_sales'] = y_pred_test
forecast_df['predicted_sales'] = forecast_df['predicted_sales'].clip(lower=0).round(0).astype(int)
forecast_df['absolute_error'] = abs(forecast_df['actual_sales'] - forecast_df['predicted_sales'])
forecast_df['percentage_error'] = (forecast_df['absolute_error'] / forecast_df['actual_sales'].replace(0, 1) * 100).round(2)

# Додавання фази
forecast_df['phase'] = 'Phase 3'

print('✅ Прогноз підготовлено!')
print(f'\nПерші рядки прогнозу:')
print(forecast_df.head(10))

In [ ]:
# Збереження прогнозу у CSV
output_filename = 'pcs_three_phase_forecast.csv'
forecast_df.to_csv(output_filename, index=False)

print(f'✅ Прогноз збережено у файл: {output_filename}')
print(f'\nРозмір файлу: {forecast_df.shape}')

## Обчислення метрик (MAE, MAPE, WMAPE)

In [ ]:
# Метрики для Phase 2 (Validation)
mae_val = mean_absolute_error(y_val, y_pred_val)
mse_val = mean_squared_error(y_val, y_pred_val)
rmse_val = np.sqrt(mse_val)

mask_val = y_val != 0
mape_val = np.mean(np.abs((y_val[mask_val] - y_pred_val[mask_val]) / y_val[mask_val])) * 100
wmape_val = (np.sum(np.abs(y_val - y_pred_val)) / np.sum(np.abs(y_val))) * 100

print('='*60)
print('МЕТРИКИ ДЛЯ PHASE 2 (VALIDATION)')
print('='*60)
print(f'MAE (Mean Absolute Error): {mae_val:.2f}')
print(f'RMSE (Root Mean Squared Error): {rmse_val:.2f}')
print(f'MAPE (Mean Absolute Percentage Error): {mape_val:.2f}%')
print(f'WMAPE (Weighted MAPE): {wmape_val:.2f}%')
print('='*60)

In [ ]:
# Метрики для Phase 3 (Test/Forecast)
y_test = phase3_data[target_column]

mae_test = mean_absolute_error(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)

mask_test = y_test != 0
mape_test = np.mean(np.abs((y_test[mask_test] - y_pred_test[mask_test]) / y_test[mask_test])) * 100
wmape_test = (np.sum(np.abs(y_test - y_pred_test)) / np.sum(np.abs(y_test))) * 100

print('='*60)
print('МЕТРИКИ ДЛЯ PHASE 3 (FORECAST)')
print('='*60)
print(f'MAE (Mean Absolute Error): {mae_test:.2f}')
print(f'RMSE (Root Mean Squared Error): {rmse_test:.2f}')
print(f'MAPE (Mean Absolute Percentage Error): {mape_test:.2f}%')
print(f'WMAPE (Weighted MAPE): {wmape_test:.2f}%')
print('='*60)

In [ ]:
# Збереження метрик у окремий CSV
metrics_data = {
    'Phase': ['Phase 2 - Validation', 'Phase 3 - Forecast'],
    'MAE': [mae_val, mae_test],
    'RMSE': [rmse_val, rmse_test],
    'MAPE': [mape_val, mape_test],
    'WMAPE': [wmape_val, wmape_test],
    'Samples': [len(y_val), len(y_test)],
    'Date': [pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')] * 2
}

metrics_df = pd.DataFrame(metrics_data)
metrics_filename = 'pcs_accuracy_calculation.csv'
metrics_df.to_csv(metrics_filename, index=False)

print(f'\n✅ Метрики збережено у файл: {metrics_filename}')
print('\nПідсумок метрик:')
print(metrics_df)

In [ ]:
# Візуалізація результатів
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Phase 2
axes[0].scatter(y_val, y_pred_val, alpha=0.5, s=10)
axes[0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[0].set_xlabel('Фактичні продажі')
axes[0].set_ylabel('Прогнозовані продажі')
axes[0].set_title(f'Phase 2 (Validation)\nWMAPE: {wmape_val:.2f}%')
axes[0].grid(True, alpha=0.3)

# Phase 3
axes[1].scatter(y_test, y_pred_test, alpha=0.5, s=10)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Фактичні продажі')
axes[1].set_ylabel('Прогнозовані продажі')
axes[1].set_title(f'Phase 3 (Forecast)\nWMAPE: {wmape_test:.2f}%')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\n🎉 Three-Phase прогнозування завершено успішно!')

In [ ]:
# Підсумок по продуктах
product_summary = forecast_df.groupby('product_id').agg({
    'actual_sales': 'sum',
    'predicted_sales': 'sum',
    'absolute_error': 'mean',
    'percentage_error': 'mean'
}).round(2)

print('\n📊 Підсумок по продуктах (Phase 3):')
print(product_summary.head(10))